In [2]:
#!pip install numpy==1.22
#!pip install pandas==1.3.4
!pip install keras==2.12.0
!pip install daal==2021.4.0

  Using cached daal-2021.4.0-py2.py3-none-manylinux1_x86_64.whl (295.5 MB)
  Using cached tbb-2021.9.0-py2.py3-none-manylinux1_x86_64.whl (4.0 MB)
  Attempting uninstall: tbb
    Found existing installation: TBB 0.2
ERROR: Cannot uninstall 'TBB'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np

2023-05-01 18:12:35.171493: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-01 18:12:35.650791: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-01 18:12:35.653209: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 18:12:38.447472: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
#download the data and read it into a pandas dataframe.
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [5]:
concrete_data.shape
concrete_data.describe()
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [6]:
#Split data into predictors and target

concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

predictors.head()
#predictors.shape
#target.shape

n_cols = predictors.shape[1] # number of predictors
n_cols


8

## Normalize Data



The next step is to normalize the data by substracting the mean and dividing by the standard deviation.


In [37]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
target_norm = (target - target.mean()) / target.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [25]:
from keras import Sequential
from tensorflow.keras import layers
from keras.layers import Dense

In [56]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

1)Randomly split the data into a training and test sets by holding 30% of the data for testing. 

In [57]:
from sklearn.model_selection import train_test_split

predictors_train, predictors_test, target_train, target_test = train_test_split(predictors_norm, target_norm, test_size=0.30,random_state=42)

#predictors_train.shape
target_test.shape
np.array(target_test).reshape(103, 3)#use reshape 103*3 = 309 original size
                                      #for the arg 'multiouput' in mean_squared_error function


array([[ 1.0231236 ,  1.20210398,  2.31549361],
       [-0.03100498, -1.51313011,  0.50653475],
       [-0.72597561,  0.57178178,  0.0947003 ],
       [ 0.78009339, -0.64157349, -1.50175677],
       [ 0.26230735,  0.83696006, -0.16808359],
       [-0.8319272 ,  0.25392699, -0.98576652],
       [ 0.08093258, -0.16329482, -0.26565484],
       [ 0.12822171,  0.77231163, -1.73101926],
       [ 0.16653189, -0.29738046, -1.47541853],
       [ 0.31318806,  1.02611658, -1.35270623],
       [ 0.98660921,  0.22639155,  0.96027096],
       [ 1.83182758, -1.24974761, -0.167485  ],
       [-0.25009133,  0.21561681, -1.32636798],
       [ 0.96625692, -1.21143743, -1.76813225],
       [ 0.13480627,  0.86329831, -1.54365854],
       [ 1.65105141,  1.1811531 ,  0.01987573],
       [-0.84210334, -1.85492876,  1.28052013],
       [ 0.71604355, -0.38357836, -1.40358694],
       [ 1.6881644 , -0.08487867, -0.62541138],
       [-1.32337499, -0.24530255, -0.96062547],
       [ 0.21561681, -1.33474833, -0.028

## Train and Test the Network


In [58]:
#Build the model

model = regression_model()


2)  Train the model on the training data using 50 epochs

In [60]:
# train the model with fit
model.fit(predictors_train, target_train, validation_split=0.3, epochs=50,verbose=2)

Epoch 1/50
16/16 - 0s - loss: 0.0945 - val_loss: 0.1464 - 200ms/epoch - 13ms/step
Epoch 2/50
16/16 - 0s - loss: 0.0939 - val_loss: 0.1451 - 113ms/epoch - 7ms/step
Epoch 3/50
16/16 - 0s - loss: 0.0932 - val_loss: 0.1444 - 106ms/epoch - 7ms/step
Epoch 4/50
16/16 - 0s - loss: 0.0935 - val_loss: 0.1444 - 103ms/epoch - 6ms/step
Epoch 5/50
16/16 - 0s - loss: 0.0932 - val_loss: 0.1442 - 106ms/epoch - 7ms/step
Epoch 6/50
16/16 - 0s - loss: 0.0934 - val_loss: 0.1446 - 106ms/epoch - 7ms/step
Epoch 7/50
16/16 - 0s - loss: 0.0918 - val_loss: 0.1437 - 135ms/epoch - 8ms/step
Epoch 8/50
16/16 - 0s - loss: 0.0920 - val_loss: 0.1427 - 108ms/epoch - 7ms/step
Epoch 9/50
16/16 - 0s - loss: 0.0908 - val_loss: 0.1442 - 127ms/epoch - 8ms/step
Epoch 10/50
16/16 - 0s - loss: 0.0916 - val_loss: 0.1427 - 124ms/epoch - 8ms/step
Epoch 11/50
16/16 - 0s - loss: 0.0905 - val_loss: 0.1441 - 130ms/epoch - 8ms/step
Epoch 12/50
16/16 - 0s - loss: 0.0900 - val_loss: 0.1436 - 107ms/epoch - 7ms/step
Epoch 13/50
16/16 - 0s -

In [61]:
#Predictions

predictions = model.predict(predictors_test)
np.array(predictions).reshape(103, 3) #use reshape 103*3 = 309 original size
                                      #for the arg 'multiouput' in mean_squared_error function


10/10 [==============================] - 0s 3ms/step


array([[ 0.5095811 ,  0.3121584 ,  1.6079769 ],
       [ 0.03846359, -1.663914  ,  0.8851115 ],
       [-0.3793136 ,  1.2272239 , -0.13492055],
       [ 0.8036676 ,  0.23989956, -1.7225201 ],
       [ 0.5017636 ,  0.37866867, -0.7348418 ],
       [-0.6777816 , -0.02557738, -1.1089429 ],
       [ 0.05956106, -0.30271083, -0.24391215],
       [-0.28147697,  0.7969994 , -1.1554075 ],
       [-0.4723121 ,  0.06918061, -1.4607456 ],
       [ 0.7449976 ,  0.87983745, -1.2793822 ],
       [ 0.6238324 , -0.22683994,  0.6646471 ],
       [ 1.6103199 , -0.9029936 ,  0.0868662 ],
       [-0.1374007 ,  0.6575269 , -1.372854  ],
       [ 1.0205393 , -1.2447017 , -1.7889767 ],
       [-0.1552407 ,  0.7647149 , -1.3188065 ],
       [ 2.2555127 ,  0.01054822,  0.20762596],
       [-0.66610056, -1.351576  ,  1.2564921 ],
       [ 0.7068842 , -0.44344032, -1.1864816 ],
       [ 0.59563786,  0.12965032, -0.6189845 ],
       [-1.4197357 ,  0.3793037 , -0.69650704],
       [ 0.6645169 , -1.4283488 ,  0.258

3) Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.


In [62]:
from sklearn.metrics import mean_squared_error

In [63]:
metrics = mean_squared_error(target_test, predictions, multioutput = 'raw_values')
metrics

array([0.14272861])

4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

In [64]:
metrics1 = np.zeros(50) #Incializando array
for i in range(49):
    model.fit(predictors_train, target_train, validation_split=0.3, epochs=50,verbose=2)
    predictions = model.predict(predictors_test)
    metrics1[i] = np.array(mean_squared_error(target_test, predictions, multioutput = 'raw_values'))
    print(metrics1[i])

Epoch 1/50
16/16 - 0s - loss: 0.0800 - val_loss: 0.1428 - 175ms/epoch - 11ms/step
Epoch 2/50
16/16 - 0s - loss: 0.0797 - val_loss: 0.1420 - 105ms/epoch - 7ms/step
Epoch 3/50
16/16 - 0s - loss: 0.0797 - val_loss: 0.1438 - 102ms/epoch - 6ms/step
Epoch 4/50
16/16 - 0s - loss: 0.0789 - val_loss: 0.1426 - 109ms/epoch - 7ms/step
Epoch 5/50
16/16 - 0s - loss: 0.0795 - val_loss: 0.1410 - 107ms/epoch - 7ms/step
Epoch 6/50
16/16 - 0s - loss: 0.0793 - val_loss: 0.1404 - 110ms/epoch - 7ms/step
Epoch 7/50
16/16 - 0s - loss: 0.0776 - val_loss: 0.1412 - 107ms/epoch - 7ms/step
Epoch 8/50
16/16 - 0s - loss: 0.0795 - val_loss: 0.1422 - 111ms/epoch - 7ms/step
Epoch 9/50
16/16 - 0s - loss: 0.0784 - val_loss: 0.1409 - 119ms/epoch - 7ms/step
Epoch 10/50
16/16 - 0s - loss: 0.0786 - val_loss: 0.1409 - 109ms/epoch - 7ms/step
Epoch 11/50
16/16 - 0s - loss: 0.0780 - val_loss: 0.1424 - 107ms/epoch - 7ms/step
Epoch 12/50
16/16 - 0s - loss: 0.0779 - val_loss: 0.1437 - 111ms/epoch - 7ms/step
Epoch 13/50
16/16 - 0s -

10/10 [==============================] - 0s 3ms/step
0.13937087883838065
Epoch 1/50
16/16 - 0s - loss: 0.0625 - val_loss: 0.1465 - 205ms/epoch - 13ms/step
Epoch 2/50
16/16 - 0s - loss: 0.0593 - val_loss: 0.1357 - 138ms/epoch - 9ms/step
Epoch 3/50
16/16 - 0s - loss: 0.0595 - val_loss: 0.1414 - 116ms/epoch - 7ms/step
Epoch 4/50
16/16 - 0s - loss: 0.0598 - val_loss: 0.1403 - 122ms/epoch - 8ms/step
Epoch 5/50
16/16 - 0s - loss: 0.0593 - val_loss: 0.1370 - 119ms/epoch - 7ms/step
Epoch 6/50
16/16 - 0s - loss: 0.0593 - val_loss: 0.1376 - 118ms/epoch - 7ms/step
Epoch 7/50
16/16 - 0s - loss: 0.0586 - val_loss: 0.1375 - 118ms/epoch - 7ms/step
Epoch 8/50
16/16 - 0s - loss: 0.0591 - val_loss: 0.1427 - 123ms/epoch - 8ms/step
Epoch 9/50
16/16 - 0s - loss: 0.0593 - val_loss: 0.1350 - 131ms/epoch - 8ms/step
Epoch 10/50
16/16 - 0s - loss: 0.0605 - val_loss: 0.1448 - 144ms/epoch - 9ms/step
Epoch 11/50
16/16 - 0s - loss: 0.0596 - val_loss: 0.1382 - 134ms/epoch - 8ms/step
Epoch 12/50
16/16 - 0s - loss: 0.

Epoch 50/50
16/16 - 0s - loss: 0.0506 - val_loss: 0.1340 - 106ms/epoch - 7ms/step
10/10 [==============================] - 0s 3ms/step
0.13834107519250655
Epoch 1/50
16/16 - 0s - loss: 0.0508 - val_loss: 0.1399 - 197ms/epoch - 12ms/step
Epoch 2/50
16/16 - 0s - loss: 0.0506 - val_loss: 0.1351 - 111ms/epoch - 7ms/step
Epoch 3/50
16/16 - 0s - loss: 0.0515 - val_loss: 0.1345 - 113ms/epoch - 7ms/step
Epoch 4/50
16/16 - 0s - loss: 0.0503 - val_loss: 0.1413 - 109ms/epoch - 7ms/step
Epoch 5/50
16/16 - 0s - loss: 0.0517 - val_loss: 0.1338 - 119ms/epoch - 7ms/step
Epoch 6/50
16/16 - 0s - loss: 0.0514 - val_loss: 0.1338 - 118ms/epoch - 7ms/step
Epoch 7/50
16/16 - 0s - loss: 0.0505 - val_loss: 0.1417 - 114ms/epoch - 7ms/step
Epoch 8/50
16/16 - 0s - loss: 0.0500 - val_loss: 0.1349 - 114ms/epoch - 7ms/step
Epoch 9/50
16/16 - 0s - loss: 0.0503 - val_loss: 0.1372 - 108ms/epoch - 7ms/step
Epoch 10/50
16/16 - 0s - loss: 0.0505 - val_loss: 0.1334 - 110ms/epoch - 7ms/step
Epoch 11/50
16/16 - 0s - loss: 0.

Epoch 49/50
16/16 - 0s - loss: 0.0448 - val_loss: 0.1254 - 108ms/epoch - 7ms/step
Epoch 50/50
16/16 - 0s - loss: 0.0448 - val_loss: 0.1339 - 101ms/epoch - 6ms/step
10/10 [==============================] - 0s 3ms/step
0.13974214944627722
Epoch 1/50
16/16 - 0s - loss: 0.0450 - val_loss: 0.1267 - 174ms/epoch - 11ms/step
Epoch 2/50
16/16 - 0s - loss: 0.0435 - val_loss: 0.1291 - 101ms/epoch - 6ms/step
Epoch 3/50
16/16 - 0s - loss: 0.0439 - val_loss: 0.1280 - 103ms/epoch - 6ms/step
Epoch 4/50
16/16 - 0s - loss: 0.0437 - val_loss: 0.1319 - 103ms/epoch - 6ms/step
Epoch 5/50
16/16 - 0s - loss: 0.0448 - val_loss: 0.1307 - 106ms/epoch - 7ms/step
Epoch 6/50
16/16 - 0s - loss: 0.0456 - val_loss: 0.1341 - 106ms/epoch - 7ms/step
Epoch 7/50
16/16 - 0s - loss: 0.0461 - val_loss: 0.1278 - 106ms/epoch - 7ms/step
Epoch 8/50
16/16 - 0s - loss: 0.0438 - val_loss: 0.1256 - 117ms/epoch - 7ms/step
Epoch 9/50
16/16 - 0s - loss: 0.0437 - val_loss: 0.1311 - 110ms/epoch - 7ms/step
Epoch 10/50
16/16 - 0s - loss: 0.

Epoch 48/50
16/16 - 0s - loss: 0.0405 - val_loss: 0.1290 - 111ms/epoch - 7ms/step
Epoch 49/50
16/16 - 0s - loss: 0.0408 - val_loss: 0.1326 - 116ms/epoch - 7ms/step
Epoch 50/50
16/16 - 0s - loss: 0.0413 - val_loss: 0.1314 - 113ms/epoch - 7ms/step
10/10 [==============================] - 0s 3ms/step
0.1380470587980325
Epoch 1/50
16/16 - 0s - loss: 0.0398 - val_loss: 0.1265 - 185ms/epoch - 12ms/step
Epoch 2/50
16/16 - 0s - loss: 0.0407 - val_loss: 0.1322 - 106ms/epoch - 7ms/step
Epoch 3/50
16/16 - 0s - loss: 0.0402 - val_loss: 0.1278 - 107ms/epoch - 7ms/step
Epoch 4/50
16/16 - 0s - loss: 0.0409 - val_loss: 0.1335 - 109ms/epoch - 7ms/step
Epoch 5/50
16/16 - 0s - loss: 0.0401 - val_loss: 0.1244 - 119ms/epoch - 7ms/step
Epoch 6/50
16/16 - 0s - loss: 0.0402 - val_loss: 0.1289 - 108ms/epoch - 7ms/step
Epoch 7/50
16/16 - 0s - loss: 0.0394 - val_loss: 0.1279 - 110ms/epoch - 7ms/step
Epoch 8/50
16/16 - 0s - loss: 0.0397 - val_loss: 0.1298 - 108ms/epoch - 7ms/step
Epoch 9/50
16/16 - 0s - loss: 0.0

...

Epoch 32/50
16/16 - 0s - loss: 0.0245 - val_loss: 0.1331 - 116ms/epoch - 7ms/step
Epoch 33/50
16/16 - 0s - loss: 0.0249 - val_loss: 0.1311 - 114ms/epoch - 7ms/step
Epoch 34/50
16/16 - 0s - loss: 0.0263 - val_loss: 0.1335 - 113ms/epoch - 7ms/step
Epoch 35/50
16/16 - 0s - loss: 0.0280 - val_loss: 0.1324 - 114ms/epoch - 7ms/step
Epoch 36/50
16/16 - 0s - loss: 0.0258 - val_loss: 0.1335 - 117ms/epoch - 7ms/step
Epoch 37/50
16/16 - 0s - loss: 0.0259 - val_loss: 0.1325 - 115ms/epoch - 7ms/step
Epoch 38/50
16/16 - 0s - loss: 0.0257 - val_loss: 0.1310 - 116ms/epoch - 7ms/step
Epoch 39/50
16/16 - 0s - loss: 0.0246 - val_loss: 0.1311 - 116ms/epoch - 7ms/step
Epoch 40/50
16/16 - 0s - loss: 0.0251 - val_loss: 0.1358 - 117ms/epoch - 7ms/step
Epoch 41/50
16/16 - 0s - loss: 0.0248 - val_loss: 0.1281 - 113ms/epoch - 7ms/step
Epoch 42/50
16/16 - 0s - loss: 0.0252 - val_loss: 0.1346 - 115ms/epoch - 7ms/step
Epoch 43/50
16/16 - 0s - loss: 0.0253 - val_loss: 0.1299 - 115ms/epoch - 7ms/step
Epoch 44/50
16/1

Epoch 31/50
16/16 - 0s - loss: 0.0244 - val_loss: 0.1300 - 115ms/epoch - 7ms/step
Epoch 32/50
16/16 - 0s - loss: 0.0247 - val_loss: 0.1319 - 117ms/epoch - 7ms/step
Epoch 33/50
16/16 - 0s - loss: 0.0262 - val_loss: 0.1335 - 116ms/epoch - 7ms/step
Epoch 34/50
16/16 - 0s - loss: 0.0251 - val_loss: 0.1300 - 122ms/epoch - 8ms/step
Epoch 35/50
16/16 - 0s - loss: 0.0263 - val_loss: 0.1328 - 113ms/epoch - 7ms/step
Epoch 36/50
16/16 - 0s - loss: 0.0247 - val_loss: 0.1297 - 112ms/epoch - 7ms/step
Epoch 37/50
16/16 - 0s - loss: 0.0251 - val_loss: 0.1270 - 117ms/epoch - 7ms/step
Epoch 38/50
16/16 - 0s - loss: 0.0239 - val_loss: 0.1314 - 114ms/epoch - 7ms/step
Epoch 39/50
16/16 - 0s - loss: 0.0242 - val_loss: 0.1310 - 117ms/epoch - 7ms/step
Epoch 40/50
16/16 - 0s - loss: 0.0247 - val_loss: 0.1326 - 116ms/epoch - 7ms/step
Epoch 41/50
16/16 - 0s - loss: 0.0258 - val_loss: 0.1336 - 112ms/epoch - 7ms/step
Epoch 42/50
16/16 - 0s - loss: 0.0258 - val_loss: 0.1346 - 112ms/epoch - 7ms/step
Epoch 43/50
16/1

Epoch 30/50
16/16 - 0s - loss: 0.0246 - val_loss: 0.1316 - 110ms/epoch - 7ms/step
Epoch 31/50
16/16 - 0s - loss: 0.0238 - val_loss: 0.1298 - 112ms/epoch - 7ms/step
Epoch 32/50
16/16 - 0s - loss: 0.0241 - val_loss: 0.1290 - 110ms/epoch - 7ms/step
Epoch 33/50
16/16 - 0s - loss: 0.0240 - val_loss: 0.1298 - 111ms/epoch - 7ms/step
Epoch 34/50
16/16 - 0s - loss: 0.0244 - val_loss: 0.1283 - 109ms/epoch - 7ms/step
Epoch 35/50
16/16 - 0s - loss: 0.0237 - val_loss: 0.1306 - 108ms/epoch - 7ms/step
Epoch 36/50
16/16 - 0s - loss: 0.0237 - val_loss: 0.1275 - 110ms/epoch - 7ms/step
Epoch 37/50
16/16 - 0s - loss: 0.0249 - val_loss: 0.1282 - 113ms/epoch - 7ms/step
Epoch 38/50
16/16 - 0s - loss: 0.0236 - val_loss: 0.1299 - 109ms/epoch - 7ms/step
Epoch 39/50
16/16 - 0s - loss: 0.0246 - val_loss: 0.1265 - 118ms/epoch - 7ms/step
Epoch 40/50
16/16 - 0s - loss: 0.0246 - val_loss: 0.1323 - 111ms/epoch - 7ms/step
Epoch 41/50
16/16 - 0s - loss: 0.0250 - val_loss: 0.1277 - 122ms/epoch - 8ms/step
Epoch 42/50
16/1

Epoch 29/50
16/16 - 0s - loss: 0.0240 - val_loss: 0.1258 - 111ms/epoch - 7ms/step
Epoch 30/50
16/16 - 0s - loss: 0.0242 - val_loss: 0.1282 - 108ms/epoch - 7ms/step
Epoch 31/50
16/16 - 0s - loss: 0.0240 - val_loss: 0.1261 - 120ms/epoch - 8ms/step
Epoch 32/50
16/16 - 0s - loss: 0.0242 - val_loss: 0.1326 - 108ms/epoch - 7ms/step
Epoch 33/50
16/16 - 0s - loss: 0.0251 - val_loss: 0.1313 - 113ms/epoch - 7ms/step
Epoch 34/50
16/16 - 0s - loss: 0.0261 - val_loss: 0.1286 - 117ms/epoch - 7ms/step
Epoch 35/50
16/16 - 0s - loss: 0.0250 - val_loss: 0.1272 - 127ms/epoch - 8ms/step
Epoch 36/50
16/16 - 0s - loss: 0.0232 - val_loss: 0.1263 - 124ms/epoch - 8ms/step
Epoch 37/50
16/16 - 0s - loss: 0.0234 - val_loss: 0.1264 - 109ms/epoch - 7ms/step
Epoch 38/50
16/16 - 0s - loss: 0.0234 - val_loss: 0.1259 - 114ms/epoch - 7ms/step
Epoch 39/50
16/16 - 0s - loss: 0.0231 - val_loss: 0.1277 - 107ms/epoch - 7ms/step
Epoch 40/50
16/16 - 0s - loss: 0.0238 - val_loss: 0.1277 - 110ms/epoch - 7ms/step
Epoch 41/50
16/1

Epoch 28/50
16/16 - 0s - loss: 0.0246 - val_loss: 0.1275 - 126ms/epoch - 8ms/step
Epoch 29/50
16/16 - 0s - loss: 0.0237 - val_loss: 0.1253 - 121ms/epoch - 8ms/step
Epoch 30/50
16/16 - 0s - loss: 0.0240 - val_loss: 0.1264 - 112ms/epoch - 7ms/step
Epoch 31/50
16/16 - 0s - loss: 0.0235 - val_loss: 0.1262 - 112ms/epoch - 7ms/step
Epoch 32/50
16/16 - 0s - loss: 0.0234 - val_loss: 0.1247 - 117ms/epoch - 7ms/step
Epoch 33/50
16/16 - 0s - loss: 0.0242 - val_loss: 0.1236 - 116ms/epoch - 7ms/step
Epoch 34/50
16/16 - 0s - loss: 0.0232 - val_loss: 0.1266 - 114ms/epoch - 7ms/step
Epoch 35/50
16/16 - 0s - loss: 0.0245 - val_loss: 0.1279 - 112ms/epoch - 7ms/step
Epoch 36/50
16/16 - 0s - loss: 0.0264 - val_loss: 0.1314 - 116ms/epoch - 7ms/step
Epoch 37/50
16/16 - 0s - loss: 0.0240 - val_loss: 0.1276 - 122ms/epoch - 8ms/step
Epoch 38/50
16/16 - 0s - loss: 0.0244 - val_loss: 0.1237 - 117ms/epoch - 7ms/step
Epoch 39/50
16/16 - 0s - loss: 0.0239 - val_loss: 0.1263 - 113ms/epoch - 7ms/step
Epoch 40/50
16/1

In [53]:
#array with 
metrics1

array([0.17743803, 0.17750239, 0.17767006, 0.17702655, 0.17771727,
       0.17740586, 0.17699917, 0.17621416, 0.17731263, 0.17799022,
       0.17716189, 0.17742532, 0.17651434, 0.17625924, 0.1767526 ,
       0.17705068, 0.17655079, 0.1775005 , 0.17898285, 0.17761174,
       0.17720337, 0.17674784, 0.1767491 , 0.17763935, 0.1785387 ,
       0.17732283, 0.17789375, 0.17665885, 0.17722433, 0.17745274,
       0.17605192, 0.17796398, 0.17848805, 0.17709717, 0.17872782,
       0.17725755, 0.17762289, 0.17630339, 0.17820042, 0.1775264 ,
       0.17791383, 0.17685687, 0.1761254 , 0.17744202, 0.17758789,
       0.17634051, 0.17693957, 0.17811984, 0.17763582, 0.        ])

5. Report the mean and the standard deviation of the mean squared errors.

In [65]:
#The std and mean values

from statistics import mean

metrics1_avg = mean(metrics1)
metrics1_std = np.std(metrics1)

print("The mean of the MSN array is: ", metrics1_avg, " and the standard deviation is: ", metrics1_std)


The mean of the MSN array is:  0.1488719489778427  and the standard deviation is:  0.022361514112905517
